In [ ]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.0/200.0 kB 7.7 MB/s eta 0:00:00


In [ ]:
#import the genai library
from google import genai

In [ ]:
#step 2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
#step 3: Get model details
model_name = "models/gemini-2.5-flash-preview-05-20"
try:
    model_details = client.models.get(model=model_name) #
    print(f"Details for model '{model_name}':")
    print(f"Model Name: {model_details.name}")
    print(f"Input Token Limit: {model_details.input_token_limit}")
    print(f"Output Token Limit: {model_details.output_token_limit}")
except Exception as e:
    print(f"Error retrieving model details for '{model_name}': {e}")

Details for model 'models/gemini-2.5-flash-preview-05-20':
Model Name: models/gemini-2.5-flash-preview-05-20
Input Token Limit: 1048576
Output Token Limit: 65536


In [ ]:
def get_weather(location: str) -> dict:
  """Get the current weather in a specific location.

  Args:
    location: The city and state, e.g. San Francisco, CA
  Returns:
    A dictionary containing the weather information.
  """
  # (Implementation of weather fetching logic would go here)
  # for now, just returning the value hard coded
  return {"temperature": "70F", "condition": "Sunny"}

# Infer schema from the Python function
get_weather_declaration_inferred = genai.types.FunctionDeclaration.from_callable(
    callable=get_weather,
    client=client
)

# Create a Tool object containing the inferred function declaration
weather_tool = genai.types.Tool(function_declarations=[get_weather_declaration_inferred])

In [ ]:
# Step 1: Send user prompt to the model
prompt = "What is the weather in New York City?"

# Use structured content with role and parts
contents = [
    genai.types.Content(role="user", parts=[genai.types.Part(text=prompt)])
]

config = genai.types.GenerateContentConfig(tools=[weather_tool])

response = client.models.generate_content(
    model=model_name,
    contents=contents,
    config=config
)

# Step 2: Extract function call
function_call_part = response.candidates[0].content.parts[0]  # assumes one part

function_name = function_call_part.function_call.name
args = function_call_part.function_call.args

print(f"Executing {function_name} with args: {args}")

# Step 3: Simulate API result
api_result = {"status": "success", "data": f"Weather in NYC is sunny"}  # placeholder

# Step 4: Create full conversation history including function call and response
conversation = [
    genai.types.Content(role="user", parts=[genai.types.Part(text=prompt)]),
    genai.types.Content(role="model", parts=[function_call_part]),
    genai.types.Content(
        role="function",
        parts=[
            genai.types.Part.from_function_response(
                name=function_name,
                response=api_result
            )
        ]
    ),
]

# Step 5: Send full conversation including function response to the model
response = client.models.generate_content(
    model=model_name,
    contents=conversation,
    config=config
)

# Step 6: Get model's final response
print(response.text)


Executing get_weather with args: {'location': 'New York, New York'}
The weather in NYC is sunny.


In [ ]:
from google.genai import types

# Define a function that the model can call to control smart lights
set_light_values_declaration = {
    "name": "set_light_values",
    "description": "Sets the brightness and color temperature of a light.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "integer",
                "description": "Light level from 0 to 100. Zero is off and 100 is full brightness",
            },
            "color_temp": {
                "type": "string",
                "enum": ["daylight", "cool", "warm"],
                "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.",
            },
        },
        "required": ["brightness", "color_temp"],
    },
}

# This is the actual function that would be called based on the model's suggestion
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {"brightness": brightness, "colorTemperature": color_temp}


In [ ]:
#declare find_flights and book_hotel functions
def find_flights(origin: str, destination: str, departure_date: str, return_date: str) -> dict:
    """Search for available flights based on origin, destination, and dates.

    Args:
        origin: Departure city or airport code (e.g., 'JFK')
        destination: Arrival city or airport code (e.g., 'LAX')
        departure_date: Departure date in YYYY-MM-DD format
        return_date: Optional return date in YYYY-MM-DD format

    Returns:
        A dictionary containing the flight search results.


    """

    #return dummy results for fligts as a list
    return [
          {
              "airline": "Delta",
              "flight_number": "123",
              "departure_time": "12:00",
              "arrival_time": "13:30",
              "price": 150.0
          },
          {
              "airline": "United",
              "flight_number": "456",
              "departure_time": "14:00",
              "arrival_time": "15:45",
              "price": 200
          }

    ]

def book_hotel(city: str, check_in: str, check_out: str, guests: int) -> dict:
    """Book a hotel room with city, dates,
    and guest count.

    Args:
        city: City where the hotel is located
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        guests: Number of guests staying

    Returns:
        A dictionary containing the hotel booking details.
    """

    #return dummy conformation with booking id
    return {"booking_id": "123456789"}


In [ ]:
#You can inspect the function declarations using the FunctionDeclaration class
#But the python SDK automatically does this for you when you use tools list

import json
from google.genai import types # Ensure types is imported

find_flights_declaration = types.FunctionDeclaration.from_callable(
    callable = find_flights,
    client = client,
)

book_hotel_declaration = types.FunctionDeclaration.from_callable(
    callable = book_hotel,
    client = client,
)

#verify
print(json.dumps(find_flights_declaration.to_json_dict(), indent=2))
print(json.dumps(book_hotel_declaration.to_json_dict(), indent=2))


{
  "description": "Search for available flights based on origin, destination, and dates.\n\nArgs:\n    origin: Departure city or airport code (e.g., 'JFK')\n    destination: Arrival city or airport code (e.g., 'LAX')\n    departure_date: Departure date in YYYY-MM-DD format\n    return_date: Optional return date in YYYY-MM-DD format\n\nReturns:\n    A dictionary containing the flight search results.",
  "name": "find_flights",
  "parameters": {
    "properties": {
      "origin": {
        "type": "STRING"
      },
      "destination": {
        "type": "STRING"
      },
      "departure_date": {
        "type": "STRING"
      },
      "return_date": {
        "type": "STRING"
      }
    },
    "required": [
      "origin",
      "destination",
      "departure_date",
      "return_date"
    ],
    "type": "OBJECT"
  }
}
{
  "description": "Book a hotel room with city, dates,\nand guest count.\n\nArgs:\n    city: City where the hotel is located\n    check_in: Check-in date in YYYY-MM-

In [ ]:
# Send request with function declarations
MANUALLY_CALL_FUNCTIONS = True #Set to True, if you want to manually call functions
chat = client.chats.create(
    model=model_name,
    config= {
        "tools" : [find_flights, book_hotel],
        "automatic_function_calling" : {"disable": MANUALLY_CALL_FUNCTIONS}
    }
)

response = chat.send_message(
    "Find flights to Paris (CDG) from San Francisco (SFO) leaving SFO on the 13th June, 2025 and returning on the 19th June, 2025; Pick the cheapest flight and then book hotel for 2.",
)

print(response.candidates[0].content.parts[0].text)

if response.candidates[0].content.parts[0].function_call:
  print(response.candidates[0].content.parts[0].function_call)


None
id=None args={'destination': 'CDG', 'origin': 'SFO', 'return_date': '2025-06-19', 'departure_date': '2025-06-13'} name='find_flights'
